## requirements

In [ ]:
!pip install transformers -U
!pip install sentencepiece -U

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Mount google drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Import libraries

In [ ]:
import pandas as pd
import numpy as np

import numpy as np
import pandas as pd
pd.set_option('display.max_colwidth', None)

import string
import re

import torch
from torch import nn
from torch.utils.data import Dataset
from tqdm import tqdm
torch.cuda.empty_cache()

import random

# huggingface
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, MT5Tokenizer, MT5ForConditionalGeneration


## Download datad from google drive

### download and unzip BBC, tebyan and pn-summary 

In [ ]:
!gdown --id 1Tpjqn9QA9FF_nqZjZZi0aLnwx3GZizfN

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1Tpjqn9QA9FF_nqZjZZi0aLnwx3GZizfN
To: /content/pn_data.zip
100% 72.6M/72.6M [00:00<00:00, 252MB/s]


In [ ]:
!gdown --id 1D8QCMWirr_fd0_n6NnDyskmMUaZxqmyJ

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1D8QCMWirr_fd0_n6NnDyskmMUaZxqmyJ
To: /content/Tebyan_data.zip
100% 172M/172M [00:02<00:00, 72.6MB/s]


In [ ]:
!unzip Tebyan_data.zip

Archive:  Tebyan_data.zip
  inflating: Tebyan_test.csv         
  inflating: Tebyan_train.csv        
  inflating: Tebyan_val.csv          


In [ ]:
!gdown --id 1-l_Oda2YBQTC4C7O7B_Cb--oLSKdlW0k

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1-l_Oda2YBQTC4C7O7B_Cb--oLSKdlW0k
To: /content/BBC_data.zip
100% 89.1M/89.1M [00:01<00:00, 72.9MB/s]


In [ ]:
!unzip BBC_data.zip

Archive:  BBC_data.zip
  inflating: BBC_train.csv           
  inflating: BBC_val.csv             
  inflating: BBC_test.csv            


In [ ]:
!unzip pn_data.zip

Archive:  pn_data.zip
replace pn_test.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace pn_train.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace pn_val.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n


In [ ]:
train_df = pd.read_csv('pn_train.csv')
test_df = pd.read_csv('pn_test.csv')
val_df = pd.read_csv('pn_val.csv')

### shape of the data

In [ ]:
train_df.shape

(82022, 3)

### Head of the data

In [ ]:
train_df.head(5)

,Unnamed: 0,Abstract,Titel
0,0,به گزارش شانا، علی کاردر امروز (۲۷ دی ماه) در مراسم تودیع محسن قمصری، مدیر سابق امور بین الملل شرکت ملی نفت ایران و معارفه سعید خوشرو، مدیر جدید امور بین الملل این شرکت، گفت: مدیریت امور بین الملل به عنوان یکی از تاثیرگذارترین مدیریت های شرکت ملی نفت ایران در دوران تحریم های ظالمانه غرب علیه کشورمان بسیار هوشمندانه عمل کرد و ما توانستیم به خوبی از عهده تحریم ها برآییم. وی افزود: مجموعه امور بین الملل در همه دوران ها با سختی ها و مشکلات بسیاری مواجه بوده است، به ویژه در دوره اخیر به دلیل مسایل پیرامون تحریم وظیفه سنگینی بر عهده داشت که با تدبیر مدیریت خوب این مجموعه سربلند از آن بیرون آمد. کاردر با قدردانی از زحمات محسن قمصری، به سلامت مدیریت امور بین الملل این شرکت اشاره کرد و افزود: محوریت کار مدیریت اموربین الملل سلامت مالی بوده است. وی بر ضرورت نهادینه سازی جوانگرایی در مدیریت شرکت ملی نفت ایران تاکید کرد و گفت: مدیریت امور بین الملل در پرورش نیروهای زبده و کارآزموده آنچنان قوی عملکرده است که برای انتخاب مدیر جدید مشکلی وجود نداشت. کاردر، حرفه ای گری و کار استاندارد را از ویژگی های مدیران این مدیریت برشمرد و گفت: نگاه جامع، خلاقیت و نوآوری و بکارگیری نیروهای جوان باید همچنان مد نظر مدیریت جدید امور بین الملل شرکت ملی نفت ایران باشد.,مدیرعامل شرکت ملی نفت، عملکرد مدیریت امور بین الملل این شرکت را در دوران تحریم بسیار هوشمندانه خواند و گفت: امور بین الملل در دوران پس از تحریم ها نیز می تواند نقش بزرگی در تسریع روند توسعه داشته باشد.
1,1,به گزارش شانا به نقل از شرکت ملی صنایع پتروشیمی، علی اصغر گودرزی فراهانی با اشاره به اینکه همه طرح های در حال اجرای صنعت پتروشیمی براساس پیشرفت فیزیکی و پیش بینی زمان راه اندازی در قالب طرح های جهش دوم و سوم تقسیم بندی شده اند، اظهار کرد: انتظار داریم که طرح های جهش دوم صنعت پتروشیمی که پیشرفت های بسیار خوبی نیز دارند تا پایان سال ۱۴۰۰ و در واقع تا پایان برنامه ششم توسعه به نتیجه برسند که براساس آن ظرفیت تولید صنعت پتروشیمی به ۱۰۰ میلیون تن در سال می رسد. وی با اشاره به پیشرفت بسیار مطلوب بعضی از طرح های جهش سوم صنعت پتروشیمی، افزود: براساس برنامه ریزی انجام شده تا پایان سال ۱۴۰۴، جهش سومی در مقدار تولید محصولات پتروشیمی رخ می دهد که با تحقق آن ظرفیت تولید کحصولات پتروشیمی به سالانه ۱۳۳ میلیون تن خواهد رسید. سرپرست مدیریت برنامه ریزی و توسعه شرکت ملی صنایع پتروشیمی با اشاره به اینکه تنوع محصولات پتروشیمی پس از بهره برداری از طرح های جهش دوم و سوم پتروشیمی افزایش قابل ملاحظه ای خواهد داشت، ادامه داد: تامین نیاز داخل در اولویت شرکت ملی صنایع پتروشیمی است و پس از تحقق این جهش ها می توان محصولات وارداتی با مصرف بالا را در داخل تولید کرد. گودرزی فراهانی تصریح کرد: شرکت ملی صنایع پتروشیمی افزون بر دریافت خوراک از بالادست و تکمیل زنجیره و جلوگیری از خام فروشی، برای متنوع کردن سبد محصولات صنعت پتروشیمی و تکمیل زنجیره ارزش نیز گام برمی دارد. وی با اشاره به اینکه تعداد محصولات اصلی تولیدی از ۹۰ محصول کنونی به ۱۰۵ محصول در پایان سال ۱۴۰۴ می رسد، گفت:: با متنوع کردن محصولات سبد پتروشیمی، مواد اولیه مورد نیاز صنایع پایین دست پتروشیمی نیز تامین می شود و و نیاز داخل به بعضی از محصولات مانند آکریلونیتریل که هم اکنون وارد می شود، تامین خواهد شد. سرپرست مدیریت برنامه ریزی و توسعه شرکت ملی صنایع پتروشیمی با اشاره به برنامه های این شرکت برای مجتمع های پتروشیمی، گفت: ۲ برنامه برای مجتمع های پتروشیمی در دستور کار است؛ نخست تلاش می شود تولید مجتمع ها به ظرفیت اسمی برسند، دو اینکه از طرح های توسعه ای و ساخت واحدهای جدید با خوراک تولیدی خود مجتمع ها استقبال می شود و برای صدور مجوز اجرای آنها سختگیری نخواهد شد.,سرپرست مدیریت برنامه ریزی و توسعه شرکت ملی صنایع پتروشیمی گفت: تنوع محصولات پتروشیمی ایران با بهره برداری از طرح های جهش دوم و سوم صنعت پتروشیمی افزایش می یابد.
2,2,به گزارش شانا به نقل از شرکت پالایش گاز شهید هاشمی نژاد، جمعی از کارشناسان این پالایشگاه با هدف معرفی محصول کود گوگرد بنتونیتی به کشاورزان در شانزدهمین نمایشگاه تخصصی کشاورزی، دام، طیور، ماشین آلات و صنایع وابسته در اردبیل حضور پیدا کردند و به معرفی دستاوردهای این شرکت پرداختند. بر اساس این گزارش، دشت مغان قطب کشاورزی در منطقه اردبیل است و بیش از ۸۵ هزار هکتار زمین کشاورزی دارد که همه آنها نیاز به استفاده از کود گوگرد بنتونیتی دارند، به همین دلیل، کارشناسان پالایشگاه گاز هاشمی نژاد در راستای توسعه بازار و معرفی محصولات 

## Summary Generation

In [ ]:
# method for summary generation, using the global model and tokenizer
def generate_summary(model, abstract, num_beams = 2, repetition_penalty = 2.5,
                    length_penalty = 2.0, early_stopping = True, max_output_length = 100):
    source_encoding=tokenizer(abstract, max_length=1000, padding="max_length", truncation=True, return_attention_mask=True, add_special_tokens=True, return_tensors="pt")

    generated_ids=model.generate(
        input_ids=source_encoding["input_ids"],
        attention_mask=source_encoding["attention_mask"],
        num_beams=num_beams,
        max_length=max_output_length,
        repetition_penalty=repetition_penalty,
        length_penalty=length_penalty,
        early_stopping=early_stopping,
        use_cache=True
        )

    # TODO ...map to string using tokenizer.decode and return
    preds=[tokenizer.decode(gen_id, skip_special_tokens=True, clean_up_tokenization_spaces=True) 
         for gen_id in generated_ids]

    return "".join(preds)

## Train and fine-tune

In [ ]:
class ThesisDataset(Dataset):
    def __init__(self, df, tokenizer, max_input_len, max_output_len):
        self.tokenizer = tokenizer
        self.source_len = max_input_len
        self.summ_len = max_output_len
        self.Titel = df.Titel

        # T5 requires us to prepend the task
        self.Abstract = 'summarize: ' + df.Abstract

    def __len__(self):
        return len(self.Titel)

    def __getitem__(self, index):
        abstract = str(self.Abstract[index])
        title = str(self.Titel[index])

        source_tok = self.tokenizer.batch_encode_plus([abstract], max_length= self.source_len, pad_to_max_length=True,return_tensors='pt')
        label_tok = self.tokenizer.batch_encode_plus([title], max_length= self.summ_len, pad_to_max_length=True,return_tensors='pt')

        input_ids = source_tok['input_ids'].squeeze()
        input_mask = source_tok['attention_mask'].squeeze()
        label_ids = label_tok['input_ids'].squeeze()
        label_mask = label_tok['attention_mask'].squeeze()

        return {
            'input_ids': input_ids.to(dtype=torch.long), 
            'input_mask': input_mask.to(dtype=torch.long), 
            'label_ids': label_ids.to(dtype=torch.long),
            'label_mask': label_mask.to(dtype=torch.long)
        }

In [ ]:
# for each point in the data loader, compute the forward pass, loss and
# backward pass

def train(epoch, tokenizer, model, device, loader, optimizer):
    model.train()

    for i, data in tqdm(enumerate(loader, 0)):
        y = data['label_ids'].to(device, dtype=torch.long)
        y_ids = y[:, :-1].contiguous()
        lm_labels = y[:, 1:].clone().detach()

        # set the padding symbols to -100 to be ignored by torch
        lm_labels[y[:, 1:] == tokenizer.pad_token_id] = -100

        inputs = data['input_ids'].to(device, dtype=torch.long)
        mask = data['input_mask'].to(device, dtype=torch.long)

        outputs = model(input_ids = inputs, attention_mask = mask, decoder_input_ids=y_ids, labels=lm_labels)

        loss = outputs[0]

        if i % 100 == 0:
            print({"Training Loss": loss.item()})
  
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    print(f'Epoch: {epoch}, Loss:  {loss.item()}')

## Validation

In [ ]:
# for validation, set the model to eval mode and compute all predictions
def validate(epoch, tokenizer, model, device, loader,num_beams = 2,max_output_length = 120,repetition_penalty = 1.0,length_penalty = 2.0,early_stopping = True):
    model.eval()
    predictions = []
    actuals = []
    with torch.no_grad():
        for i, data in enumerate(loader, 0):
            y = data['label_ids'].to(device, dtype=torch.long)
            ids = data['input_ids'].to(device, dtype=torch.long)
            mask = data['input_mask'].to(device, dtype=torch.long)

            generated_ids = model.generate(input_ids=ids,
                attention_mask=mask,
                num_beams=num_beams,
                max_length=max_output_length,
                repetition_penalty=repetition_penalty,
                length_penalty=length_penalty,
                early_stopping=early_stopping,
                use_cache=True)
            
            preds=[tokenizer.decode(gen_id, skip_special_tokens=True, clean_up_tokenization_spaces=True) for gen_id in generated_ids]
            target = [tokenizer.decode(gen_id, skip_special_tokens=True, clean_up_tokenization_spaces=True) for gen_id in y]
            if i % 100 == 0:
                print(f'Completed {i}')

            predictions.extend(preds)
            actuals.extend(target)
    
    return predictions, actuals

## Clear the cache

In [ ]:
import gc
import torch
gc.collect()
torch.cuda.empty_cache()

In [ ]:
!gdown --id 1HUcxb7_j4AWTr5tjmnP5f6lmvCx-3bV1

In [ ]:
# define tokenizer
tokenizer = MT5Tokenizer.from_pretrained("google/mt5-small")

#load our previous model from google drive
model = MT5ForConditionalGeneration.from_pretrained("summerization_8_16/mt5-small-fine-tune-10")

device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = model.to(device)

# Defining the optimizer that will be used to tune the weights of the network in the training session. 
optimizer = torch.optim.Adam(params=model.parameters(), lr=1e-4)

## Defining some parameters

In [ ]:
# defining some parameters that will be used later on in the training  
batch_size_train = 6
batch_size_vali = 2

max_input_len = 1000   
max_output_len = 100

# set random seeds and deterministic pytorch for reproducibility
seed = 42
torch.manual_seed(seed)
np.random.seed(seed)
torch.backends.cudnn.deterministic = True
    
# split the dataframe into training and validation
df_train = train_df
df_vali = val_df


# df_train = df.sample(frac=0.8, random_state=seed)
# df_vali = df.drop(df_train.index).reset_index(drop=True)
df_train = df_train.reset_index(drop=True)
print(f"train={df_train.shape}, vali={df_vali.shape}")

# Creating the Training and Validation dataset for further creation of Dataloader
ds_train = ThesisDataset(df_train, tokenizer, max_input_len, max_output_len)
ds_vali = ThesisDataset(df_vali, tokenizer, max_input_len, max_output_len)

# create data loaders for training and validation
from torch.utils.data import DataLoader
dl_train = DataLoader(ds_train, shuffle=True, num_workers=0, batch_size=batch_size_train)
dl_vali = DataLoader(ds_vali, shuffle=True, num_workers=0, batch_size=batch_size_vali)

train=(77672, 3), vali=(6865, 3)


### number of the iteration

In [ ]:
len(dl_train)

12946

## start training...

In [ ]:
epochs_train = 1
rouge_list = []
# epoch = 1
for epoch in range(epochs_train):
    train(epoch=epoch, tokenizer=tokenizer, model=model, device=device, loader=dl_train, optimizer=optimizer)
    # save our model into the google drive
    model.save_pretrained('drive/MyDrive/summerization_8_17/mt5-small-fine-tune-'+ str(epoch))
    print('end of Epoch:'+ str(epoch))
    predictions, actuals = validate(epoch, tokenizer, model, device, dl_vali)

0it [00:00, ?it/s]Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2329: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


{'Training Loss': 3.00264310836792}


100it [01:18,  1.28it/s]

{'Training Loss': 1.4207717180252075}


200it [02:39,  1.26it/s]

...

{'Training Loss': 1.738150715827942}


12900it [2:51:59,  1.25it/s]

{'Training Loss': 2.3350982666015625}


12946it [2:52:35,  1.25it/s]


Epoch: 0, Loss:  1.5978379249572754
end of Epoch:0


## Load model and compare outputs

In [ ]:
!gdown --id 1eMO3p1pzjh27Gbt0GYX896Ai1dbVXpFv

In [ ]:
# load model and compare outputs
fine = AutoModelForSeq2SeqLM.from_pretrained('summerization_8_17/mt5-small-fine-tune-0')

### Select 500 index randomly from test Data

In [ ]:
thesis_picks = [random.randint(0, len(test_df) - 1) for _ in range(500)]

In [ ]:
!mkdir drive/MyDrive/thesis_picks

### save inex in google drive

In [ ]:
with open(f'/content/drive/My Drive/thesis_picks/thesis.txt', "w") as f:
  for t in thesis_picks:
    f.write(f"{t}\n")

## Evaluation

### pn-summary

In [ ]:
from IPython.display import HTML, display

# adjust these values as desired
num_beams = 2
repetition_penalty = 1.0
length_penalty = 2.0
max_output_length = 120
# min_output_length = 20
early_stopping = True


predictions_2 =[]
references_2 =[]

for num, i in tqdm(enumerate(thesis_picks)):
    summarize = test_df.iloc[i].Abstract
    reference = test_df.iloc[i].Titel

    s2 = generate_summary(fine, summarize, num_beams, repetition_penalty,
                        	    length_penalty, early_stopping, max_output_length)
    predictions_2.append(s2)
    references_2.append(reference)

500it [58:23,  7.01s/it]


In [ ]:
predictions_2[13]

'برای خروج از مشکلات نیازمند وحدت ملی و دوری از تفرقه و شخصی نگری است و نباید تفرقه در بین مردم و مسیولان ایجاد شود.'

### requirment for evaluation

In [ ]:
!pip install rouge -q
!pip install evaluate -q
!pip install bert_score -q

     |████████████████████████████████| 69 kB 6.2 MB/s 
     |████████████████████████████████| 141 kB 36.1 MB/s 
     |████████████████████████████████| 365 kB 56.0 MB/s 
     |████████████████████████████████| 212 kB 58.9 MB/s 
     |████████████████████████████████| 115 kB 63.2 MB/s 
     |████████████████████████████████| 127 kB 58.9 MB/s 
     |████████████████████████████████| 60 kB 6.3 MB/s 


### Rough for pn-summary

In [ ]:
from rouge import Rouge 
rouge = Rouge()
scores = rouge.get_scores(predictions_2, references_2)

In [ ]:
rough_dic = {}
rough_dic['rouge-1'] = []
rough_dic['rouge-2'] = []
rough_dic['rouge-l'] = []
for score in scores:
  for k,v in score.items():
    rough_dic[k].append(v['f'])

In [ ]:
r_df = pd.DataFrame(rough_dic)
r_df.head()

,rouge-1,rouge-2,rouge-l
0,0.258065,0.000000,0.258065
1,0.457143,0.315789,0.457143
2,0.444444,0.285714,0.444444
3,0.500000,0.370370,0.500000
4,0.521739,0.382979,0.391304


In [ ]:
r_df.mean()

rouge-1    0.316142
rouge-2    0.146439
rouge-l    0.262791
dtype: float64

### BERTScore for pn-summary

In [ ]:
from evaluate import load
bertscore = load("bertscore")
results = bertscore.compute(predictions=predictions_2, references=references_2, lang="fa")

In [ ]:
b_df = pd.DataFrame(results)
b_df.head()

,precision,recall,f1,hashcode
0,0.731924,0.782924,0.756565,bert-base-multilingual-cased_L9_no-idf_version=0.3.11(hug_trans=4.21.1)
1,0.791285,0.750256,0.770224,bert-base-multilingual-cased_L9_no-idf_version=0.3.11(hug_trans=4.21.1)
2,0.848902,0.750790,0.796837,bert-base-multilingual-cased_L9_no-idf_version=0.3.11(hug_trans=4.21.1)
3,0.849400,0.757432,0.800784,bert-base-multilingual-cased_L9_no-idf_version=0.3.11(hug_trans=4.21.1)
4,0.811244,0.774176,0.792277,bert-base-multilingual-cased_L9_no-idf_version=0.3.11(hug_trans=4.21.1)


In [ ]:
b_df.mean()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  """Entry point for launching an IPython kernel.


precision    0.755349
recall       0.722182
f1           0.737647
dtype: float64

### BBC evaluation

In [ ]:
test_df = pd.read_csv('BBC_test.csv')

In [ ]:
thesis_picks = [random.randint(0, len(test_df) - 1) for _ in range(500)]

### rough for BBC

In [ ]:
predictions_3 =[]
references_3 =[]

for num, i in tqdm(enumerate(thesis_picks)):
    summarize = test_df.iloc[i].Abstract
    reference = test_df.iloc[i].Titel

    s2 = generate_summary(fine, summarize, num_beams, repetition_penalty,
                        	    length_penalty, early_stopping, max_output_length)
    predictions_3.append(s2)
    references_3.append(reference)

500it [57:06,  6.85s/it]


In [ ]:
scores_2 = rouge.get_scores(predictions_3, references_3)
rough_dic = {}
rough_dic['rouge-1'] = []
rough_dic['rouge-2'] = []
rough_dic['rouge-l'] = []
for score in scores_2:
  for k,v in score.items():
    rough_dic[k].append(v['f'])

In [ ]:
r_df = pd.DataFrame(rough_dic)
r_df.head()

,rouge-1,rouge-2,rouge-l
0,0.097561,0.0,0.097561
1,0.098361,0.0,0.065574
2,0.047619,0.0,0.047619
3,0.263158,0.0,0.263158
4,0.140351,0.0,0.105263


In [ ]:
r_df.mean()

rouge-1    0.229804
rouge-2    0.061780
rouge-l    0.184868
dtype: float64

### BERSTScore for BBC

In [ ]:
results = bertscore.compute(predictions=predictions_3, references=references_3, lang="fa")

In [ ]:
b_df = pd.DataFrame(results)
b_df.head()

,precision,recall,f1,hashcode
0,0.735286,0.698309,0.716320,bert-base-multilingual-cased_L9_no-idf_version=0.3.11(hug_trans=4.21.1)
1,0.725772,0.633542,0.676528,bert-base-multilingual-cased_L9_no-idf_version=0.3.11(hug_trans=4.21.1)
2,0.746697,0.671074,0.706869,bert-base-multilingual-cased_L9_no-idf_version=0.3.11(hug_trans=4.21.1)
3,0.741826,0.728742,0.735226,bert-base-multilingual-cased_L9_no-idf_version=0.3.11(hug_trans=4.21.1)
4,0.834574,0.772307,0.802234,bert-base-multilingual-cased_L9_no-idf_version=0.3.11(hug_trans=4.21.1)


In [ ]:
b_df.mean()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  """Entry point for launching an IPython kernel.


precision    0.727697
recall       0.695398
f1           0.710623
dtype: float64

### Merge BBC and Pn-summary

In [ ]:
BBC_test = pd.read_csv('BBC_test.csv')
pn_test = pd.read_csv('pn_test.csv')
all = pd.concat([BBC_test, pn_test])

In [ ]:
thesis_picks = [random.randint(0, len(all) - 1) for _ in range(500)]

In [ ]:
predictions_4 =[]
references_4 =[]

for num, i in tqdm(enumerate(thesis_picks)):
    summarize = all.iloc[i].Abstract
    reference = all.iloc[i].Titel

    s2 = generate_summary(fine, summarize, num_beams, repetition_penalty,
                        	    length_penalty, early_stopping, max_output_length)
    predictions_4.append(s2)
    references_4.append(reference)

500it [57:17,  6.88s/it]


### Rough of the merged data

In [ ]:
scores_2 = rouge.get_scores(predictions_4, references_4)
rough_dic = {}
rough_dic['rouge-1'] = []
rough_dic['rouge-2'] = []
rough_dic['rouge-l'] = []
for score in scores_2:
  for k,v in score.items():
    rough_dic[k].append(v['f'])

In [ ]:
r_df = pd.DataFrame(rough_dic)
r_df.head()

,rouge-1,rouge-2,rouge-l
0,0.000000,0.000000,0.000000
1,0.388889,0.333333,0.388889
2,0.222222,0.043478,0.177778
3,0.122449,0.000000,0.081633
4,0.186047,0.097561,0.186047


In [ ]:
r_df.mean()

rouge-1    0.255364
rouge-2    0.086656
rouge-l    0.203886
dtype: float64

### BERTScore of the merged data

In [ ]:
results = bertscore.compute(predictions=predictions_4, references=references_4, lang="fa")

In [ ]:
b_df = pd.DataFrame(results)
b_df.head()

,precision,recall,f1,hashcode
0,0.688342,0.613180,0.648591,bert-base-multilingual-cased_L9_no-idf_version=0.3.11(hug_trans=4.21.1)
1,0.824616,0.754269,0.787875,bert-base-multilingual-cased_L9_no-idf_version=0.3.11(hug_trans=4.21.1)
2,0.721762,0.654822,0.686664,bert-base-multilingual-cased_L9_no-idf_version=0.3.11(hug_trans=4.21.1)
3,0.723224,0.658656,0.689431,bert-base-multilingual-cased_L9_no-idf_version=0.3.11(hug_trans=4.21.1)
4,0.759624,0.668823,0.711338,bert-base-multilingual-cased_L9_no-idf_version=0.3.11(hug_trans=4.21.1)


In [ ]:
b_df.mean()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  """Entry point for launching an IPython kernel.


precision    0.737733
recall       0.700449
f1           0.717984
dtype: float64